In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.4 MB/s eta 0:00:00


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_value_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_value_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_type_value_counts[application_type_value_counts<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_value_counts = application_df['CLASSIFICATION'].value_counts()
classification_value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications_to_replace = list(classification_value_counts[classification_value_counts <1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_df = pd.get_dummies(application_df)
dummy_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [9]:
# Split our preprocessed data into our features and target arrays
y = dummy_df['IS_SUCCESSFUL']
x = dummy_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=1, stratify=y)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [16]:
input_features = len(x_train_scaled[0])
nn_model = tf.keras.models.Sequential()
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid', 'leaky_relu'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=100,
        step=2), activation=activation, input_dim= input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 40)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [17]:
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2)

In [18]:
tuner.search(x_train_scaled,y_train,epochs=50,validation_data=(x_test_scaled,y_test))

Trial 52 Complete [00h 03m 28s]
val_accuracy: 0.729912519454956

Best val_accuracy So Far: 0.7318950295448303
Total elapsed time: 00h 26m 59s

Search: Running Trial #53

Value             |Best Value So Far |Hyperparameter
sigmoid           |sigmoid           |activation
41                |77                |first_units
13                |2                 |num_layers
3                 |7                 |units_0
9                 |1                 |units_1
3                 |1                 |units_2
1                 |3                 |units_3
9                 |9                 |units_4
5                 |7                 |units_5
1                 |5                 |units_6
3                 |9                 |units_7
9                 |7                 |units_8
5                 |5                 |units_9
3                 |7                 |units_10
5                 |7                 |units_11
5                 |5                 |units_12
9                 |9        

KeyboardInterrupt: 

In [19]:
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5710 - accuracy: 0.7319 - 1s/epoch - 5ms/step
Loss: 0.5709826350212097, Accuracy: 0.7318950295448303


In [20]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'first_units': 77,
 'num_layers': 2,
 'units_0': 7,
 'units_1': 1,
 'units_2': 1,
 'units_3': 3,
 'units_4': 9,
 'units_5': 7,
 'units_6': 5,
 'units_7': 9,
 'units_8': 7,
 'units_9': 5,
 'units_10': 7,
 'units_11': 7,
 'units_12': 5,
 'units_13': 9,
 'units_14': 9,
 'units_15': 1,
 'units_16': 3,
 'units_17': 3,
 'units_18': 1,
 'units_19': 1,
 'units_20': 3,
 'units_21': 9,
 'units_22': 1,
 'units_23': 7,
 'units_24': 3,
 'units_25': 5,
 'units_26': 7,
 'units_27': 3,
 'tuner/epochs': 50,
 'tuner/initial_epoch': 17,
 'tuner/bracket': 3,
 'tuner/round': 3,
 'tuner/trial_id': '0046',
 'units_28': 7,
 'units_29': 3,
 'units_30': 3,
 'units_31': 7,
 'units_32': 7,
 'units_33': 1,
 'units_34': 3,
 'units_35': 5,
 'units_36': 7}

In [22]:
# Split our preprocessed data into our features and target arrays
y = dummy_df['IS_SUCCESSFUL']
x = dummy_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=1, test_size=0.2, stratify=y)

In [23]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(x_train_scaled[0])
nn_model3 = tf.keras.models.Sequential()

# First hidden layer
nn_model3.add(tf.keras.layers.Dense(units=80, input_dim = input_features, activation = 'relu'))

# second hidden layer
nn_model3.add(tf.keras.layers.Dense(units=30, activation = 'relu'))

# Output layer
nn_model3.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn_model3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 80)                3520      
                                                                 
 dense_5 (Dense)             (None, 30)                2430      
                                                                 
 dense_6 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
nn_model3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
nn_model3.fit(x_train_scaled, y_train, epochs=50)

Epoch 1/50
858/858 [==============================] - 3s 2ms/step - loss: 0.5711 - accuracy: 0.7184
Epoch 2/50
858/858 [==============================] - 3s 3ms/step - loss: 0.5566 - accuracy: 0.7262
Epoch 3/50
858/858 [==============================] - 2s 2ms/step - loss: 0.5540 - accuracy: 0.7271
Epoch 4/50
858/858 [==============================] - 2s 2ms/step - loss: 0.5521 - accuracy: 0.7302
Epoch 5/50
858/858 [==============================] - 2s 2ms/step - loss: 0.5505 - accuracy: 0.7300
Epoch 6/50
858/858 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7295
Epoch 7/50
858/858 [==============================] - 2s 2ms/step - loss: 0.5488 - accuracy: 0.7307
Epoch 8/50
858/858 [==============================] - 4s 4ms/step - loss: 0.5483 - accuracy: 0.7313
Epoch 9/50
858/858 [==============================] - 2s 2ms/step - loss: 0.5476 - accuracy: 0.7309
Epoch 10/50
858/858 [==============================] - 2s 2ms/step - loss: 0.5470 - accuracy: 0.7317

In [31]:
model_loss, model_accuracy = nn_model3.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5528 - accuracy: 0.7366 - 399ms/epoch - 2ms/step
Loss: 0.5527751445770264, Accuracy: 0.7365888953208923


In [35]:
# Split our preprocessed data into our features and target arrays
y = dummy_df['IS_SUCCESSFUL']
x = dummy_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=1, test_size=0.15, stratify=y)

In [36]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [37]:
input_features = len(x_train_scaled[0])
nn_model4 = tf.keras.models.Sequential()

# First hidden layer
nn_model4.add(tf.keras.layers.Dense(units=80, input_dim = input_features, activation = 'relu'))

#second hidden layer
nn_model4.add(tf.keras.layers.Dense(units=60, activation = 'relu'))
# Second hidden layer
nn_model4.add(tf.keras.layers.Dense(units=30, activation = 'relu'))

# Output layer
nn_model4.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn_model4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 80)                3520      
                                                                 
 dense_12 (Dense)            (None, 60)                4860      
                                                                 
 dense_13 (Dense)            (None, 30)                1830      
                                                                 
 dense_14 (Dense)            (None, 1)                 31        
                                                                 
Total params: 10241 (40.00 KB)
Trainable params: 10241 (40.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
nn_model4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [39]:
nn_model4.fit(x_train_scaled, y_train, epochs=50)

Epoch 1/50
912/912 [==============================] - 7s 4ms/step - loss: 0.5698 - accuracy: 0.7210
Epoch 2/50
912/912 [==============================] - 4s 4ms/step - loss: 0.5554 - accuracy: 0.7273
Epoch 3/50
912/912 [==============================] - 3s 3ms/step - loss: 0.5527 - accuracy: 0.7304
Epoch 4/50
912/912 [==============================] - 2s 2ms/step - loss: 0.5506 - accuracy: 0.7315
Epoch 5/50
912/912 [==============================] - 2s 2ms/step - loss: 0.5498 - accuracy: 0.7317
Epoch 6/50
912/912 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7316
Epoch 7/50
912/912 [==============================] - 3s 4ms/step - loss: 0.5486 - accuracy: 0.7332
Epoch 8/50
912/912 [==============================] - 3s 3ms/step - loss: 0.5477 - accuracy: 0.7333
Epoch 9/50
912/912 [==============================] - 2s 2ms/step - loss: 0.5470 - accuracy: 0.7332
Epoch 10/50
912/912 [==============================] - 2s 2ms/step - loss: 0.5466 - accuracy: 0.7332

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model4.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

161/161 - 0s - loss: 0.5531 - accuracy: 0.7329 - 354ms/epoch - 2ms/step
Loss: 0.5531414747238159, Accuracy: 0.7329446077346802
